In [1]:
! pip3 install -q findspark
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext,SparkConf

In [3]:
conf=SparkConf().setAppName("test").setMaster("local[*]")
sc=SparkContext(conf=conf)
sc

<SparkContext master=local[*] appName=test>

### RDD

In [4]:
import random
randomlist=random.sample(range(0,40),10)
print(randomlist)

[32, 14, 31, 12, 3, 22, 27, 24, 21, 9]


In [5]:
#Create RDD
rdd1=sc.parallelize(randomlist,4)
rdd1

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:262

In [6]:
rdd1.collect()

[32, 14, 31, 12, 3, 22, 27, 24, 21, 9]

In [7]:
rdd2=sc.parallelize([1,14,20,20,29,10,13,3],2)
rdd2.collect()

[1, 14, 20, 20, 29, 10, 13, 3]

### Union

In [9]:
rdd_union=rdd1.union(rdd2)
rdd_union.collect()

[27, 16, 32, 22, 36, 33, 10, 26, 14, 18, 1, 14, 20, 20, 29, 10, 13, 3]

### intersect

In [8]:
rdd_intersect=rdd1.intersection(rdd2)
rdd_intersect.collect()

[14, 3]

In [9]:
rdd_intersect.getNumPartitions()

6

In [10]:
#Find the Empty Partition
print(rdd_intersect.glom().collect())
count=0
for item in rdd_intersect.glom().collect():
    if len(item) == 0:
        count+=1
count

[[], [], [14], [3], [], []]


4

### coalese

In [11]:
#Coalese
rdd_intersect.coalesce(1).collect()

[14, 3]

In [12]:
rdd_intersect.glom().collect()

[[], [], [14], [3], [], []]

In [13]:
rdd1.collect()

[32, 14, 31, 12, 3, 22, 27, 24, 21, 9]

In [14]:
#Take Sample (with replacement,num)
rdd1.takeSample(False,5)

[32, 21, 9, 22, 14]

In [15]:
#Take order
print(rdd1.takeOrdered(5))
print(rdd1.takeOrdered(5,key=lambda x:-x))

[3, 9, 12, 14, 21]
[32, 31, 27, 24, 22]


### Reduce

In [16]:
#Reduce
rdd1.reduce(lambda x,y:x+y)

195

### ReduceByKey

In [17]:
#ReduceByKey
rdd_rbk=sc.parallelize([(1,4),(1,2),(22,2),(6,3),(22,5),(5,2),(6,7)],2)
print(rdd_rbk.glom().collect())
import pandas as pd
countter = pd.DataFrame({"key":rdd_rbk.keys().collect(),"value":rdd_rbk.values().collect()})
print(countter)
print(rdd_rbk.reduceByKey(lambda x,y:x+y).collect())

[[(1, 4), (1, 2), (22, 2)], [(6, 3), (22, 5), (5, 2), (6, 7)]]
   key  value
0    1      4
1    1      2
2   22      2
3    6      3
4   22      5
5    5      2
6    6      7
[(22, 7), (6, 10), (1, 6), (5, 2)]


### SortByKey

In [18]:
#sortByKey()
rdd_rbk.reduceByKey(lambda x,y:x+y).sortByKey().collect()

[(1, 6), (5, 2), (6, 10), (22, 7)]

### CountByKey

In [19]:
#countByKey()
rdd_rbk.countByKey()

defaultdict(int, {1: 2, 22: 2, 6: 2, 5: 1})

### GroupKey

In [20]:
#groupbykey
rdd_group = rdd_rbk.groupByKey()
rdd_group.getNumPartitions()

print(rdd_group.collect())

for item in rdd_group.collect():
    print(item[0],[values for values in item[1]])


[(22, <pyspark.resultiterable.ResultIterable object at 0x7fbf1da05610>), (6, <pyspark.resultiterable.ResultIterable object at 0x7fbf1da056d0>), (1, <pyspark.resultiterable.ResultIterable object at 0x7fbf1da058e0>), (5, <pyspark.resultiterable.ResultIterable object at 0x7fbf1da05850>)]
22 [2, 5]
6 [3, 7]
1 [4, 2]
5 [2]


### lookup

In [54]:
#lookup key

rdd_rbk.lookup(7)

[]

### Cache memory = save the data in cache memory

In [21]:
# cache
rdd_rbk.persist()

ParallelCollectionRDD[19] at readRDDFromFile at PythonRDD.scala:262

### Persist

In [22]:
# Persisrence
from pyspark import StorageLevel
rdd1.persist(StorageLevel.MEMORY_AND_DISK)

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:262

In [23]:
#note:
# searh about the persistance